# Local Spatial Functions

Local spatial functions compute a variety of spatial information on a local, per tile basis, meaning that every tile is evaluated independently. `withCenter`, `withCenterLatLng`, `withBounds`, and `withSpatialIndex` all create a new column in the rasterframe titled `center`, `center`, `bounds`, and `spatial_index` respectively containing the results of the query.

We do our usual imports and set up the sparksession

In [1]:
from pyrasterframes import *
from pyrasterframes.rasterfunctions import *
import pyspark
from pyspark.sql import SparkSession
from pathlib import Path


spark = SparkSession.builder. \
    master("local[*]"). \
    appName("RasterFrames"). \
    config("spark.ui.enabled", "false"). \
    getOrCreate(). \
    withRasterFrames()

We define our function to process the tiff.

In [2]:
resource_dir = Path('./samples').resolve()
filenamePattern = "L8-B{}-Elkton-VA.tiff"
bandNumbers = range(1, 5)

def readTiff(band):
    return spark.read.geotiff(resource_dir.joinpath(filenamePattern.format(band)).as_uri())

In [3]:
from functools import reduce
joinedRF = reduce(lambda rf1, rf2: rf1.asRF().spatialJoin(rf2.drop('bounds').drop('metadata')),
                  [readTiff(b).withColumnRenamed('tile', 'band_{}'.format(b)) for b in bandNumbers])

`withCenter` creates a column containing the center of the tiles in the native CRS.

In [4]:
joinedRF.withCenter().select("center").show(truncate = False)

+---------------------------------------+
|center                                 |
+---------------------------------------+
|POINT (706767.7980160001 4252076.74344)|
+---------------------------------------+



`withCenterLatLng` performs a similar function, but displays the centers in longitude, latitude form (EPSG:4326).

In [5]:
joinedRF.withCenterLatLng().select("center").show(truncate = False)

+--------------------------------------+
|center                                |
+--------------------------------------+
|[-78.63240544923785,38.39301867116852]|
+--------------------------------------+



`withBounds` creates a column containing the polygonal bounds of the tile in the native CRS.

In [6]:
joinedRF.withBounds().select("bounds").show(truncate = False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|bounds                                                                                                                                                   |
+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|POLYGON ((703986.502389 4249551.61978, 703986.502389 4254601.8671, 709549.093643 4254601.8671, 709549.093643 4249551.61978, 703986.502389 4249551.61978))|
+---------------------------------------------------------------------------------------------------------------------------------------------------------+



`withSpatialIndex` creates a spatial index column

In [7]:
joinedRF.withSpatialIndex().select("spatial_index").show(truncate = False)

+-------------------+
|spatial_index      |
+-------------------+
|2781571893406213788|
+-------------------+

